In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
!pip install keras_metrics
import keras_metrics
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
%cd /content/
## split text file ##
with open('min-ind3.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
import sklearn
from sklearn.model_selection import train_test_split
train_val,test = train_test_split(lines,test_size=0.3) # here split size can be change with test_size parameter
with open('min-ind_test.txt','w',encoding='utf-8') as w:
  for line in test:
    w.write(line+'\n')

with open('min-ind_train.txt','w',encoding='utf-8') as w:
  for line in train_val:
    w.write(line+'\n')

kf = KFold(n_splits=10) # 10 folds for training and validation sets
save_dir = '/content/'
fold_var = 10
batch_size = 128
epochs = 10
latent_dim = 256 # edit LSTM units
num_samples = 14032
data_path = 'min-ind_train.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
  lines = f.read().split("\n")
  #print('lines',lines)
print('lines',lines[:len(lines)])
print(len(lines))
for line in lines[:num_samples]:
  if len(line.split(','))==2:
    input_text, target_text = line.split(",")
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character.
  target_text = "\t" + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
      if char not in input_characters:
          input_characters.add(char)
  for char in target_text:
      if char not in target_characters:
          target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

/content
lines ['pamaliharoan,pemeliharaan', 'bisnis,bisnis', 'binomial,binomial', 'perwira tinggi,jenderal', 'bendera,bendera', 'malaysia,malaysia', 'mandau,mandau', 'pangalola,pengelola', 'contohnya,contohnya', 'periodik,periodik', 'penerbit,penerbit', 'tagak,diri', 'lambok,lembab', 'kasiah,kasih', 'takana,teringat', 'pambarian,pemberian', 'enam,enam', 'garotak,ancaman', 'spinosus,spinosus', 'elongata,elongata', 'alang,alang', 'angel,angel', 'tantunya,tentunya', 'galia,curang', 'marasioan,merahasiakan', 'balatih,berlatih', 'jala,jala', 'gabon,gabon', 'kawasan,kawasan', 'barisiko,beresiko', 'voli,voli', 'mangusulkan,mengusulkan', 'ganggam,genggam', 'biskuit,biskuit', 'baduri,berduri', 'buliah,boleh', 'perangkok,perangkap', 'titanium,titanium', 'tarikh,tarikh', 'sukun,sukun', 'gea,gea', 'sukoi,sukai', 'sasamo,sesama', 'kanker,kanker', 'panutan,panutan', 'padi,padi', 'putuih,putus', 'pua,pua', 'lelang,lelang', 'maksimum,maksimum', 'pangusaho,pengusaha', 'demonstran,demonstran', 'han,han

In [ ]:

###### 5-Fold Cross Validation ####

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
kf = KFold(n_splits=10) # 5 folds for training and validation sets
save_dir = '/content/'
fold_var = 10
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(input_texts)),decoder_target_data)):
  print('k={}'.format(i+1))
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.LSTM(latent_dim, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  #encoder_inputs = embedding_layer_input(encoder_inputs)
  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c] # hidden and cell state

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  #decoder_inputs = embedding_layer_output(decoder_inputs)
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)


  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  encoder_input_data_t = pd.Series(list(encoder_input_data)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data)).iloc[val_index].to_numpy()


  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])
  #checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
	#						monitor='val_accuracy', verbose=1, 
	#						save_best_only=True, mode='max')
	#callbacks_list = [checkpoint]
	# There can be other callbacks  

  #adam = tf.keras.optimizers.SGD(lr=0.001)
  x_test = [encoder_input_data_val,decoder_input_data_val]
  y_test = decoder_target_data_val
  model.compile(
    optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  results = model.fit(
      [encoder_input_data_t, decoder_input_data_t],
      decoder_target_data_t,
      batch_size=batch_size,
      epochs=epochs,
      validation_data =(x_test,y_test)
  )
  plt.plot(results.history['accuracy'])
  plt.plot(results.history['val_accuracy'])
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend(['train','val'],loc='upper right')
  plt.title('Model Accuracy')
  plt.show()
  model.save('model_lstm_256_{}-Fold{}'.format(fold_var,i+1))
  model = keras.models.load_model("model_lstm_256_{}-Fold{}".format(fold_var,i+1)) # Using 5-Fold cross validated model

  encoder_inputs = model.input[0]  # input_1
  encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
  encoder_states = [state_h_enc, state_c_enc]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  decoder_lstm = model.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )

  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence
  import tensorflow as tf
  import numpy as np
  from tensorflow import keras
  #!pip install keras_metrics
  import keras_metrics

  from sklearn.model_selection import KFold
  import numpy as np
  import pandas as pd
  %cd /content/drive/MyDrive/
  #kf = KFold(n_splits=10) # 10 folds for training and validation sets
  save_dir = '/content/'
  fold_var = 10
  batch_size = 128
  epochs = 10
  latent_dim = 256 # edit LSTM units
  num_samples = 4211
  data_path = 'min-ind_test.txt'
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    #print('lines',lines)
  print('lines',lines[:len(lines)])
  print(len(lines))
  for line in lines[:num_samples]:
    if len(line.split(','))==2:
      input_text, target_text = line.split(",")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))
  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])

  print("Number of samples:", len(input_texts))
  print("Number of unique input tokens:", num_encoder_tokens)
  print("Number of unique output tokens:", num_decoder_tokens)
  print("Max sequence length for inputs:", max_encoder_seq_length)
  print("Max sequence length for outputs:", max_decoder_seq_length)

  input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
  target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data = np.zeros(
      (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
  )
  decoder_input_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )
  decoder_target_data = np.zeros(
      (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
  )

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t, input_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
      for t, char in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep
          decoder_input_data[i, t, target_token_index[char]] = 1.0
          if t > 0:
              # decoder_target_data will be ahead by one timestep
              # and will not include the start character.
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0
  
  df = pd.DataFrame()
  input_texts_ls = []
  decoded_sentences_ls = []
  for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    input_texts_ls.append(input_texts[seq_index])
    decoded_sentences_ls.append(decoded_sentences_ls)
    #print(type(input_texts_ls))
    #print("-")
    #print("Input sentence:", input_texts[seq_index])
    #print("Decoded sentence:", decoded_sentence)
    
  
  #print(df)
  #df_csv = df.to_csv('Input_decoded_sentences_5-fold-{}.csv'.format(i))
  
  
# Save model
#model.save("s2s_1024_f")

k=1
Epoch 1/10
70/70 [==============================] - 25s 312ms/step - loss: 2.0378 - accuracy: 0.5447 - recall_1: 0.3609 - precision_1: 0.8563 - val_loss: 1.1870 - val_accuracy: 0.6761 - val_recall_1: 0.5588 - val_precision_1: 0.9885
Epoch 2/10
 6/70 [=>............................] - ETA: 17s - loss: 1.2204 - accuracy: 0.6524 - recall_1: 0.5514 - precision_1: 0.9765

In [ ]:

###### 10-Fold Cross Validation ####

from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
kf = KFold(n_splits=10) # 10 folds for training and validation sets
save_dir = '/content/drive/MyDrive/'
fold_var = 10
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(input_texts)),decoder_target_data)):
  print('K={}'.format(i+1))
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder = keras.layers.LSTM(latent_dim, return_state=True)
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  #encoder_inputs = embedding_layer_input(encoder_inputs)
  # We discard `encoder_outputs` and only keep the states.
  encoder_states = [state_h, state_c] # hidden and cell state

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  #decoder_inputs = embedding_layer_output(decoder_inputs)
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)


  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs)

  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
  encoder_input_data_t = pd.Series(list(encoder_input_data)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data)).iloc[val_index].to_numpy()


  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])
  #checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
	#						monitor='val_accuracy', verbose=1, 
	#						save_best_only=True, mode='max')
	#callbacks_list = [checkpoint]
	# There can be other callbacks  

  #adam = tf.keras.optimizers.SGD(lr=0.001)
  x_test = [encoder_input_data_val,decoder_input_data_val]
  y_test = decoder_target_data_val
  model.compile(
    optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  results = model.fit(
      [encoder_input_data_t, decoder_input_data_t],
      decoder_target_data_t,
      batch_size=batch_size,
      epochs=epochs,
      validation_data =(x_test,y_test)
  )
  plt.plot(results.history['accuracy'])
  plt.plot(results.history['val_accuracy'])
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend(['train','val'],loc='upper right')
  plt.title('Model Accuracy')
  plt.show()
  model.save('model_lstm_256_{}-Fold_{}'.format(fold_var,i+1))
  model = keras.models.load_model("model_lstm_256_{}-Fold{}".format(fold_var,i+1)) # Using 5-Fold cross validated model

  encoder_inputs = model.input[0]  # input_1
  encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
  encoder_states = [state_h_enc, state_c_enc]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  decoder_lstm = model.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )

  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence
  df = pd.DataFrame()
  input_texts_ls = []
  decoded_sentences_ls = []
  for seq_index in range(50):
      # Take one sequence (part of the training set)
      # for trying out decoding.
      input_seq = encoder_input_data[seq_index : seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      input_texts_ls.append(input_texts[seq_index])
      decoded_sentences_ls.append(decoded_sentences_ls)
      print("-")
      print("Input sentence:", input_texts[seq_index])
      print("Decoded sentence:", decoded_sentence)
 # df['Input sentences'] = input_texts_ls
 # df['Decoded sentences'] = decoded_sentences_ls
 # df.to_csv('Input_decoded_sentences_10-fold-{}.csv'.format(i))
 
  
# Save model
#model.save("s2s_1024_f")

In [ ]:
!pip install keras_metrics